In [ ]:
import pickle as pkl
import numpy as np
import pandas as pd
import anndata as ad

In [ ]:
with open("all_ECOOTL.0.01.pkl", "rb") as f:
    d = pkl.load(f)

In [ ]:
kinase_used = ad.read_h5ad("../../../../../data/chemical_screen/kinase_used.h5ad")

In [ ]:
proteins = kinase_used.obsm['adt_select_norm'].columns.tolist() + kinase_used.obsm['tsb_select_norm'].columns.tolist()

In [ ]:
genes = kinase_used.var.index.tolist()

In [ ]:
proteins = [p.split("_")[-1] for p in proteins]

In [ ]:
genes_ser = pd.Series(genes)

In [ ]:
in_genes = []
gene_idx = []
for p in proteins:
    in_genes.append(p.upper() in genes)
    if p.upper() not in genes:
        print(p.upper())
    else:
        gene_idx.append(genes.index(p.upper()))

In [ ]:
in_genes = pd.Series(in_genes)
in_genes.value_counts()

In [ ]:
shared_prot_idx = np.where(in_genes)[0]
shared_gene_idx = pd.Series(gene_idx)

In [ ]:
methods = [
    "EOT_ott",
    "LEOT_ott",
    "EGW_ott",
    "EGW_all_ott",
    "EGWL_ott",
    "ECOOT",
    "ECOOTL",
    "perfect",
    "by_conc",
    "random",
]
enrichments = []
epsilons = [0.01, 1e-3, 1e-4, 1e-05]
ms = []
epss = []
min_foscttm = []
for method in methods:
    for eps in epsilons:
        
        with open(f"features_{method}.{eps}.pkl", "rb") as f:
            d = pkl.load(f)
        ms.append(method)
        epss.append(eps)
        enrichments.append(d["Tv"][shared_prot_idx,shared_gene_idx].sum() * (d["Tv"].size/len(shared_gene_idx)))
edf = pd.DataFrame({"method":ms, "eps":epss, "enrichment":enrichments})

In [ ]:
edf.groupby('method')['enrichment'].max()